<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#00000;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Create ReactionDataset to compute TS and reaction energies </h1>
<br><br>
</center>

In [1]:
import qcportal as ptl

## Define variable name

In [2]:
client_address = "152.74.10.245:7778"
username = None
password = None 

# Collections with monomers
mon_opt_col_name = 'monomers'
monomer_name = 'd-lactide'
init_opt_col_name = 'initiators'
initiator_name = 'salen-OMe'

# Collections with the reactant products and intermediates
ri_opt_col_name = "salen-OMe_d-lac_minima"
## Collections with the tranistion states
ts_opt_col_name = "salen-OMe_d-lac_ts_from_scan"
## Name of the ReactionDataset collection
name_rx = "salen_d-lac_rxn_pbe0-d3_defs-svp"

# Names of reactants, transistion state and intermediate/product 
reac = 'r'
ts = 'ts1'
interm = 'i1'

# The optimization level of theory

lot = "pbe0-d3_def2-svp"

In [3]:
client = ptl.FractalClient(address=client_address, username = username, password = password, verify=False)

## A) Create ReactionDataset (If it does not exist yet)

In [4]:
#ds_rx = ptl.collections.ReactionDataset(name_rx, ds_type="rxn", client=client, default_program="psi4")
#ds_rx.save()

## B) Call the OptimizationDataset with the optimized states

In [81]:
ri_opt   =  client.get_collection("OptimizationDataset", ri_opt_col_name)
ts_opt   =  client.get_collection("OptimizationDataset", ts_opt_col_name)
mon_opt  =  client.get_collection("OptimizationDataset", mon_opt_col_name)
init_opt =  client.get_collection("OptimizationDataset", init_opt_col_name)

In [82]:
ri_opt.status(collapse = False)

,pbe0-d3_def2-svp
ts1_01_r,COMPLETE
ts1_01_i1,COMPLETE
ts1_03_r,COMPLETE
ts1_03_i1,COMPLETE


In [7]:
ds_rx = client.get_collection("ReactionDataset", name_rx)

## C) Create the stoichiometry 

In [87]:
mol_ids = {}
recs = ri_opt.data.records
paths = []

for path_key in recs.keys():
    if not ts in path_key:
        continue
    paths.append(path_key.split("_")[1])
    
paths = list(set(paths))
print(paths)

mon_rec = mon_opt.get_record(monomer_name, specification = lot)
opt_mol_mon = mon_rec.get_final_molecule()

initiator_rec = init_opt.get_record(initiator_name, specification = lot)
opt_mol_initiator = initiator_rec.get_final_molecule()


for p in paths:
    reac_path_name = ts+'_'+p+'_'+reac
    int_path_name =  ts+'_'+p+'_'+interm
    ts_path_name =  ts+'_'+p
    rx_entry_name =  reac+'_'+ts+'_'+interm+'_'+p

    reac_rec = ri_opt.get_record(reac_path_name, specification = lot)
    int_rec = ri_opt.get_record(int_path_name, specification = lot)
    ts_rec  = ts_opt.get_record(ts_path_name, specification = lot)
   

    if (reac_rec.status  == "ERROR" or ts_rec.status  == "ERROR" or int_rec.status  == "ERROR"):
        print("Reaction in path {} ended with an error.".format(path_key))
        continue
    
    opt_mol_int = int_rec.get_final_molecule()
    opt_mol_reac = reac_rec.get_final_molecule()
    opt_mol_ts = ts_rec.get_final_molecule()



    be_cal = {
            "default":[(opt_mol_int,1.0),(opt_mol_reac,-1.0)],
            "rx_energy" :[(opt_mol_int,1.0),(opt_mol_reac,-1.0)],
            "ts_energy" :[(opt_mol_ts,1.0),(opt_mol_reac,-1.0)],
            "rev_ts_energy" :[(opt_mol_ts,1.0),(opt_mol_int,-1.0)],
            "inter_energy" :[(opt_mol_mon,-1.0),(opt_mol_initiator,-1.0), (opt_mol_reac,1.0)],
            }
    try: 
        ds_rx.add_rxn(rx_entry_name, be_cal)
        ds_rx.save()
        print("Created reaction path {} ".format(p))
    except KeyError:
        print("Reaction path {} does already exit".format(p))
        continue

['01', '03']
Reaction path 01 does already exit
Reaction path 03 does already exit


## D) Check Records 

In [88]:
ds_rx.data.records

[ReactionEntry(attributes={}, reaction_results={'default': {}}, name='r_ts1_i1_01', stoichiometry={'default': {'44569': 1.0, '44614': -1.0}, 'rx_energy': {'44569': 1.0, '44614': -1.0}, 'ts_energy': {'43830': 1.0, '44614': -1.0}, 'rev_ts_energy': {'43830': 1.0, '44569': -1.0}, 'inter_energy': {'31275': -1.0, '38098': -1.0, '44614': 1.0}}, extras={}),
 ReactionEntry(attributes={}, reaction_results={'default': {}}, name='r_ts1_i1_03', stoichiometry={'default': {'44716': 1.0, '44769': -1.0}, 'rx_energy': {'44716': 1.0, '44769': -1.0}, 'ts_energy': {'44197': 1.0, '44769': -1.0}, 'rev_ts_energy': {'44197': 1.0, '44716': -1.0}, 'inter_energy': {'31275': -1.0, '38098': -1.0, '44769': 1.0}}, extras={})]

## E) Compute energies

In [5]:
## Define levels of theory to be computed

elot = ["B3LYP-D3BJ", "M06-L" , "PBE0-D3BJ", 'M11-L', "B3PW91-D3BJ", "M05-2X", "REVM06-L", "SCAN0", "REVSCAN0", "SCAN-D3BJ2B"]

# M11-L good performance for TS barriers: 10.1021/jz201525
# revM06-L good performance for TS barriers https://www.pnas.org/doi/10.1073/pnas.1705670114

In [90]:
# Send computations to server
stoichs = ['default', 'ts_energy', 'rev_ts_energy', 'inter_energy']
c_list = []
for s in stoichs:
    for i in elot:
        c =  ds_rx.compute(i, "def2-tzvp" , stoich=s, tag='energies')       
        print(c)
        c_list.append(c)

ComputeResponse(nsubmitted=4 nexisting=4)
ComputeResponse(nsubmitted=2 nexisting=2)
ComputeResponse(nsubmitted=4 nexisting=4)
ComputeResponse(nsubmitted=2 nexisting=2)
ComputeResponse(nsubmitted=4 nexisting=4)
ComputeResponse(nsubmitted=2 nexisting=2)
ComputeResponse(nsubmitted=2 nexisting=2)
ComputeResponse(nsubmitted=2 nexisting=2)
ComputeResponse(nsubmitted=2 nexisting=2)
ComputeResponse(nsubmitted=4 nexisting=4)
ComputeResponse(nsubmitted=2 nexisting=6)
ComputeResponse(nsubmitted=1 nexisting=3)
ComputeResponse(nsubmitted=2 nexisting=6)
ComputeResponse(nsubmitted=1 nexisting=3)
ComputeResponse(nsubmitted=2 nexisting=6)
ComputeResponse(nsubmitted=1 nexisting=3)
ComputeResponse(nsubmitted=1 nexisting=3)
ComputeResponse(nsubmitted=1 nexisting=3)
ComputeResponse(nsubmitted=1 nexisting=3)
ComputeResponse(nsubmitted=2 nexisting=6)
ComputeResponse(nsubmitted=0 nexisting=8)
ComputeResponse(nsubmitted=0 nexisting=4)
ComputeResponse(nsubmitted=0 nexisting=8)
ComputeResponse(nsubmitted=0 nexis

### Compute energies at Optimization level of theory

In [39]:
#kw = ptl.models.KeywordSet(**{"values": {"dftd": "d3", "convthre" : '3.0e-7', "threall" : '1.0e-13', "scf" : "diis+a"}})
#ds_rx.add_keywords("dft-d3", "terachem", kw)
#ds_rx.save()

In [93]:
for s in stoichs:
    c0 =  ds_rx.compute("pbe0", "def2-svp" , stoich=s,  keywords='dft-d3', tag='tera_en', program='terachem')
    c0
c0

<ComputeResponse(nsubmitted=2 nexisting=2)>

##  D) Check stoichiometry records 

In [6]:
ds_rx = client.get_collection("ReactionDataset", name_rx)
stoichs = ['default', 'ts_energy', 'rev_ts_energy', 'inter_energy']

rq_method = []

for m in elot:
    if 'D3' in m:
        rq_method.append(m.split('-')[0])
    else:
        rq_method.append(m)

for s in stoichs:
    for m in rq_method:
        print("Records for {} in stoichiometry {}".format(m, s))
        print(ds_rx.get_records(method=m, stoich=s))
        print('-----------------------------------------------')

Records for B3LYP in stoichiometry default
                                                                    record
name        stoichiometry idx                                             
r_ts1_i1_01 default       0    ResultRecord(id='54406', status='COMPLETE')
                          0    ResultRecord(id='54558', status='COMPLETE')
                          0    ResultRecord(id='54361', status='COMPLETE')
                          1    ResultRecord(id='54405', status='COMPLETE')
                          1    ResultRecord(id='54557', status='COMPLETE')
                          1    ResultRecord(id='54360', status='COMPLETE')
r_ts1_i1_03 default       0    ResultRecord(id='54625', status='COMPLETE')
                          1    ResultRecord(id='54624', status='COMPLETE')
-----------------------------------------------
Records for M06-L in stoichiometry default
                                                                    record
name        stoichiometry idx            

                                                                    record
name        stoichiometry idx                                             
r_ts1_i1_01 ts_energy     0    ResultRecord(id='54573', status='COMPLETE')
                          0    ResultRecord(id='54601', status='COMPLETE')
                          0    ResultRecord(id='54424', status='COMPLETE')
                          1    ResultRecord(id='54565', status='COMPLETE')
                          1    ResultRecord(id='54587', status='COMPLETE')
                          1    ResultRecord(id='54415', status='COMPLETE')
r_ts1_i1_03 ts_energy     0    ResultRecord(id='54657', status='COMPLETE')
                          1    ResultRecord(id='54636', status='COMPLETE')
-----------------------------------------------
Records for M05-2X in stoichiometry ts_energy
                                                                    record
name        stoichiometry idx                                             
r_ts1_

                                                                    record
name        stoichiometry idx                                             
r_ts1_i1_01 rev_ts_energy 0    ResultRecord(id='54603', status='COMPLETE')
                          1    ResultRecord(id='54592', status='COMPLETE')
r_ts1_i1_03 rev_ts_energy 0    ResultRecord(id='54659', status='COMPLETE')
                          1    ResultRecord(id='54641', status='COMPLETE')
-----------------------------------------------
Records for SCAN0 in stoichiometry rev_ts_energy
                                                                    record
name        stoichiometry idx                                             
r_ts1_i1_01 rev_ts_energy 0    ResultRecord(id='54604', status='COMPLETE')
                          1    ResultRecord(id='54594', status='COMPLETE')
r_ts1_i1_03 rev_ts_energy 0    ResultRecord(id='54660', status='COMPLETE')
                          1    ResultRecord(id='54643', status='COMPLETE')
---

                                                                    record
name        stoichiometry idx                                             
r_ts1_i1_01 inter_energy  0    ResultRecord(id='54585', status='COMPLETE')
                          0    ResultRecord(id='54610', status='COMPLETE')
                          0    ResultRecord(id='54438', status='COMPLETE')
                          1    ResultRecord(id='54586', status='COMPLETE')
                          1    ResultRecord(id='54611', status='COMPLETE')
                          1    ResultRecord(id='54439', status='COMPLETE')
                          2    ResultRecord(id='54567', status='COMPLETE')
                          2    ResultRecord(id='54589', status='COMPLETE')
                          2    ResultRecord(id='54417', status='COMPLETE')
r_ts1_i1_03 inter_energy  0    ResultRecord(id='54585', status='COMPLETE')
                          0    ResultRecord(id='54610', status='COMPLETE')
                         

## E) Get Reaction Energies

In [7]:
ds_rx.get_values(method=elot, basis='def2-tzvp', stoich='inter_energy')

,inter_energy-M11-L/def2-tzvp,inter_energy-SCAN0/def2-tzvp,inter_energy-REVM06-L/def2-tzvp,inter_energy-M05-2X/def2-tzvp,inter_energy-REVSCAN0/def2-tzvp,inter_energy-M06-L/def2-tzvp,inter_energy-PBE0-D3BJ/def2-tzvp,inter_energy-B3PW91-D3BJ/def2-tzvp,inter_energy-B3LYP-D3BJ/def2-tzvp,inter_energy-SCAN-D3BJ2B/def2-tzvp
r_ts1_i1_01,-8.854476,-9.615025,-12.412177,-11.607102,-5.085843,-13.67441,-14.054802,-15.293305,-15.367614,NaN
r_ts1_i1_03,-6.178116,-5.178681,-10.691425,-7.127821,-0.819075,-10.073531,-8.789432,-9.9535,-10.423756,NaN


In [8]:
ds_rx.get_values(method=elot, basis='def2-tzvp', stoich='ts_energy')

,ts_energy-M05-2X/def2-tzvp,ts_energy-REVSCAN0/def2-tzvp,ts_energy-SCAN0/def2-tzvp,ts_energy-M11-L/def2-tzvp,ts_energy-REVM06-L/def2-tzvp,ts_energy-M06-L/def2-tzvp,ts_energy-PBE0-D3BJ/def2-tzvp,ts_energy-B3PW91-D3BJ/def2-tzvp,ts_energy-B3LYP-D3BJ/def2-tzvp,ts_energy-SCAN-D3BJ2B/def2-tzvp
r_ts1_i1_01,10.05899,8.922712,9.712617,7.039241,10.095801,9.244574,11.231697,11.242895,12.18552,NaN
r_ts1_i1_03,5.759616,5.631335,5.828839,5.123236,7.890113,5.109018,6.539466,6.057604,7.541927,NaN


In [9]:
ds_rx.get_values(method=elot,basis='def2-tzvp', stoich='default')

,M11-L/def2-tzvp,SCAN0/def2-tzvp,REVM06-L/def2-tzvp,M05-2X/def2-tzvp,REVSCAN0/def2-tzvp,M06-L/def2-tzvp,PBE0-D3BJ/def2-tzvp,B3PW91-D3BJ/def2-tzvp,B3LYP-D3BJ/def2-tzvp,SCAN-D3BJ2B/def2-tzvp
r_ts1_i1_01,5.516626,9.371167,10.359627,9.693975,8.475351,8.810863,11.472463,11.532137,13.337345,NaN
r_ts1_i1_03,4.113824,5.429342,9.069497,5.818786,5.359192,5.192646,6.812652,6.427766,8.846442,NaN


In [15]:
ds_rx.get_values(method=elot,basis='def2-tzvp', stoich='rev_ts_energy')

,rev_ts_energy-REVM06-L/def2-tzvp,rev_ts_energy-M11-L/def2-tzvp,rev_ts_energy-SCAN0/def2-tzvp,rev_ts_energy-REVSCAN0/def2-tzvp,rev_ts_energy-M05-2X/def2-tzvp,rev_ts_energy-M06-L/def2-tzvp,rev_ts_energy-PBE0-D3BJ/def2-tzvp,rev_ts_energy-B3PW91-D3BJ/def2-tzvp,rev_ts_energy-B3LYP-D3BJ/def2-tzvp,rev_ts_energy-SCAN-D3BJ2B/def2-tzvp
r_ts1_i1_01,-0.263827,1.522615,0.341449,0.447362,0.365015,0.433711,-0.240766,-0.289242,-1.151825,NaN
r_ts1_i1_03,-1.179384,1.009412,0.399497,0.272143,-0.05917,-0.083629,-0.273186,-0.370162,-1.304515,NaN


In [100]:
ds_rx.get_values(method='pbe0', basis = 'def2-svp', stoich='ts_energy', program='terachem')

,ts_energy-PBE0/def2-svp-Terachem,ts_energy-PBE0/def2-svp-dft-d3-Terachem
r_ts1_i1_01,-39.271035,9.299528
r_ts1_i1_03,2.879888,2.879888


In [99]:
ds_rx.get_values(method='pbe0', basis = 'def2-svp',  stoich='default', program='terachem')

,PBE0/def2-svp-dft-d3-Terachem,PBE0/def2-svp-Terachem
r_ts1_i1_01,8.279794,-40.290769
r_ts1_i1_03,2.241575,2.241575


## Restart Jobs

To restart jobs you need to run compute jobs again to retrive the job ids

In [ ]:
stoichs = ds_rx.data.records[0].stoichiometry.keys()

rq_method = []

for m in elot:
    if 'D3' in m:
        rq_method.append(m.split('-')[0])
    else:
        rq_method.append(m)

for m in rq_method:
    for st in stoichs:
        all_rec =  ds_rx.get_records(method=m, stoich=st)['record']
        for r in all_rec:
            print(m, r, st)
            idd = r.dict()['id']
            if r.dict()['status'] == 'ERROR':
                restart = client.modify_tasks("restart", base_result = idd)
                print(restart)

In [21]:
restart = client.modify_tasks("restart", base_result = 54365)
restart

Data(n_updated=1)

##  Delete Entries

In [ ]:
#del ds_rx.data.records[0]
#ds_rx.save()

In [91]:
ds_rx.compute?